<a href="https://colab.research.google.com/github/Sanskar6877/ChestXRay_Pneumonia_Detection/blob/main/Model5_ui_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model, load_model
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
IMAGE_SIZE = [224, 224]
train_path = '/content/drive/MyDrive/chest_xray/train'
valid_path = '/content/drive/MyDrive/chest_xray/val'
test_path = '/content/drive/MyDrive/chest_xray/test'

In [4]:
vgg = VGG19(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

80134624/80134624 [==============================] - 2s 0us/step


In [5]:

for layer in vgg.layers:
    layer.trainable = False


In [6]:
folders = glob('/content/drive/MyDrive/chest_xray/train/*')

In [7]:
x = Flatten()(vgg.output)

In [8]:
prediction = Dense(len(folders), activation='softmax')(x)

# creating a model object
model = Model(inputs=vgg.input, outputs=prediction)

In [9]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [10]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [11]:
from keras.preprocessing.image import ImageDataGenerator

In [12]:


# Use the Image Data Generator to import the images from the dataset
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   featurewise_center=False,  # set input mean to 0 over the dataset
                                  samplewise_center=False,  # set each sample mean to 0
                                 featurewise_std_normalization=False,  # divide inputs by std of the dataset
                                 samplewise_std_normalization=False,  # divide each input by its std
                                 zca_whitening=False,  # apply ZCA whitening
                              rotation_range = 30,  # randomly rotate images in the range (degrees, 0 to 180)
                             zoom_range = 0.2, # Randomly zoom image
                            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
                           height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
                         horizontal_flip = True,  # randomly flip images
                        vertical_flip=False)

val_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen=ImageDataGenerator(rescale=1./255)

# providing the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/chest_xray/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')



val_set = val_datagen.flow_from_directory('/content/drive/MyDrive/chest_xray/val',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/chest_xray/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')



Found 5216 images belonging to 2 classes.
Found 220 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [14]:
# fiting the model

r = model.fit(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Epoch 1/5
163/163 [==============================] - 1196s 7s/step - loss: 0.2897 - accuracy: 0.8878 - val_loss: 0.2470 - val_accuracy: 0.9038
Epoch 2/5
163/163 [==============================] - 134s 820ms/step - loss: 0.1747 - accuracy: 0.9273 - val_loss: 0.3340 - val_accuracy: 0.8862
Epoch 3/5
163/163 [==============================] - 134s 820ms/step - loss: 0.1803 - accuracy: 0.9268 - val_loss: 0.2537 - val_accuracy: 0.9022
Epoch 4/5
163/163 [==============================] - 135s 827ms/step - loss: 0.1322 - accuracy: 0.9500 - val_loss: 0.2901 - val_accuracy: 0.9071
Epoch 5/5
163/163 [==============================] - 132s 809ms/step - loss: 0.1325 - accuracy: 0.9490 - val_loss: 0.6204 - val_accuracy: 0.8125


In [15]:
!pip install gradio
import gradio as gr
import cv2
import numpy as np
img_size = 224
def load_and_preprocess_image(img):
    img = np.array(img)
    img = cv2.resize(img, (img_size, img_size))
    img = img.reshape(-1, img_size, img_size, 3)
    img = img / 255.0
    return img

def image_classifier(img):
    # Preprocess the image
    img = load_and_preprocess_image(img)
    # Perform prediction
    prediction = model.predict(img)[0]  # Assuming your model outputs a single probability
    # Return the prediction
    return {'PNEUMONIA': prediction[1], 'NORMAL': prediction[0]}  # Invert the prediction for Gradio

# Create Gradio interface
demo = gr.Interface(fn=image_classifier, inputs="image", outputs="label")
# Launch the interface
demo.launch()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 62.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 12.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.4/312.4 kB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 92.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 10.5 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl s

In [16]:
test_loss, test_acc = model.evaluate(test_set, verbose=2)
print('Test accuracy:', test_acc)
print('Test loss:',test_loss)

20/20 - 14s - loss: 0.6204 - accuracy: 0.8125 - 14s/epoch - 707ms/step
Test accuracy: 0.8125
Test loss: 0.6204298138618469
